In [1]:
import torch
import torchvision
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
import numpy as np
from torch.utils.data import  Dataset,DataLoader
from PIL import Image

In [2]:
class KMNIST(Dataset):
  
  def __init__(self, train = True, root = ''):
    self.root = root
    
    self.X_train = torch.from_numpy(np.load(root+"/k49-train-imgs.npz")['arr_0']).reshape(-1,1,28,28)
    self.X_train = self.X_train / 255
    self.y_train = torch.from_numpy(np.load(root+"/k49-train-labels.npz")['arr_0'])
    self.y_train = self.y_train.type(torch.LongTensor)
    
    
    self.X_test = torch.from_numpy(np.load(root+"/k49-test-imgs.npz")['arr_0']).reshape(-1,1,28,28)
    self.X_test = self.X_test / 255
    self.y_test = torch.from_numpy(np.load(root+"/k49-test-labels.npz")['arr_0'])
    self.y_test = self.y_test.type(torch.LongTensor)
    
    self.train = train
  
  def __len__(self):
    return self.X_train.shape[0] if self.train else self.X_test.shape[0]
  
  def __getitem__(self,index):
    if self.train: 
      return self.X_train[index],self.y_train[index]
    
    else: 
      return self.X_test[index],self.y_test[index]


In [3]:
data_train1 = KMNIST(root='/content/drive/MyDrive/hiragana')
data_test2 = KMNIST(root='/content/drive/MyDrive/hiragana',train = False)

In [ ]:
data_train1[0]

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters 
num_epochs = 5
batch_size = 16
learning_rate = 0.002 

In [ ]:
train_loader = torch.utils.data.DataLoader(data_train1, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(data_test2, batch_size=batch_size,
                                         shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 49)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through fc1
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output

In [ ]:
net = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate,momentum= 0.9)


In [ ]:
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=49, bias=True)
)


In [ ]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.cuda())
        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.346
[1,  4000] loss: 1.661
[1,  6000] loss: 1.365
[1,  8000] loss: 1.162
[1, 10000] loss: 1.026
[1, 12000] loss: 0.925
[1, 14000] loss: 0.866
[2,  2000] loss: 0.775
[2,  4000] loss: 0.736
[2,  6000] loss: 0.718
[2,  8000] loss: 0.689
[2, 10000] loss: 0.653
[2, 12000] loss: 0.645
[2, 14000] loss: 0.624
[3,  2000] loss: 0.571
[3,  4000] loss: 0.570
[3,  6000] loss: 0.551
[3,  8000] loss: 0.549
[3, 10000] loss: 0.558
[3, 12000] loss: 0.523
[3, 14000] loss: 0.529
[4,  2000] loss: 0.498
[4,  4000] loss: 0.482
[4,  6000] loss: 0.472
[4,  8000] loss: 0.480
[4, 10000] loss: 0.472
[4, 12000] loss: 0.464
[4, 14000] loss: 0.474
[5,  2000] loss: 0.438
[5,  4000] loss: 0.438
[5,  6000] loss: 0.436
[5,  8000] loss: 0.428
[5, 10000] loss: 0.431
[5, 12000] loss: 0.423
[5, 14000] loss: 0.428
Finished Training


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(dtype=torch.float).cuda()
            y = y.to(dtype=torch.long).cuda()
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()

check_accuracy(test_loader, net)
check_accuracy(train_loader,net)

Checking accuracy on test data
Got 34187 / 38547 with accuracy 88.69
Checking accuracy on training data
Got 222400 / 232365 with accuracy 95.71
